In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [15]:
JupyterDash.infer_jupyter_proxy_config()

## Cargar los datos

In [4]:
import json
with open('Colombia.geo.json') as f:
    counties = json.load(f)

In [5]:
Departamentos = []
geocol = counties.copy()
for man in geocol['features']:
    Departamentos.append(man['properties']['NOMBRE_DPT'])
    man['id']=man['properties']['NOMBRE_DPT']

In [6]:
df = pd.read_csv('PIDAR_DASH.csv')
df.head()

,Unnamed: 0,BP,No. VP,Nombre Proyecto,Departamento,Municipio,Total beneficiarios,Hombres,Mujeres,Total Víctimas,Valor cofinanciado ADR,Valor encargo fiduciario,Valor total cofinanciación ADR,Valor Contrapartida,Valor total proyecto,Hectáreas,Resolución,Fecha resolución,Vigencia,Lineamiento de consejo directivo
0,0,1,1,Implementar un sistema de ganadería silvopasto...,CAQUETA,SAN VICENTE DEL CAGUÁN,46,26,20,0,534000000,NaN,534000000,115640000,649640000,40,1220,06/29/2018 12:00:00 AM,2017,NaN
1,1,2,2,Fortalecimiento del cultivo de arroz y caña pa...,NARIÑO,SAN ANDRÉS DE TUMACO,19,7,12,0,185000000,NaN,185000000,265082460,450082460,64,1234,07/06/2017 12:00:00 AM,2017,NaN
2,2,3,3,Implementar un modelo piscícola para la produc...,CAQUETA,SAN VICENTE DEL CAGUÁN,52,36,16,31,905900513,NaN,905900513,104748820,1010649333,1,1322,08/15/2017 12:00:00 AM,2017,NaN
3,3,4,4,Mejorar las capacidades productivas para la ag...,MAGDALENA,ZONA BANANERA,150,121,29,0,2948347650,NaN,2948347650,665760350,3614108000,411,1307,08/04/2017 12:00:00 AM,2017,NaN
4,4,5,5,Mejorar las capacidades productivas de 97 fami...,CAUCA,BUENOS AIRES,97,76,21,0,500000000,NaN,500000000,581106000,1081106000,98,1215,06/29/2017 12:00:00 AM,2017,NaN


In [7]:
fig = px.choropleth_mapbox(df,                                      #Data
        locations='Departamento',                              #Column containing the identifiers used in the GeoJSON file 
        color='Valor cofinanciado ADR',                        #Column giving the color intensity of the region
        geojson=geocol,                                        #The GeoJSON file
        zoom=3,                                                #Zoom
        mapbox_style="carto-positron",                         #Mapbox style, for different maps you need a Mapbox account and a token
        range_color=(0, df['Valor cofinanciado ADR'].max()), 
        center={"lat":4.7110, "lon": -74.0721},                #Center
        color_continuous_scale="Viridis",                      #Color Scheme
        opacity=0.5,                                           #Opacity of the map
        )

In [8]:
#df.loc[df['pop'] < 2.e6, 'country'] = 'Other countries' # Represent only large countries
fig2 = px.pie(df, values='Total beneficiarios', names='Departamento', title='Total de beneficiarios ADR')

In [9]:
fig3 = px.bar(df, x = 'Hectáreas', y='Departamento')

In [10]:
available_indicators = df['Municipio'].unique()

In [11]:
df['Municipio'].head()

0    SAN VICENTE DEL CAGUÁN
1      SAN ANDRÉS DE TUMACO
2    SAN VICENTE DEL CAGUÁN
3             ZONA BANANERA
4              BUENOS AIRES
Name: Municipio, dtype: object

In [ ]:
# Initialize app

app = dash.Dash(
    __name__,
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)
server = app.server

app.layout = html.Div(
    id="root",
    children=[
        html.Div(
            id="header",
            children=[
                html.Img(id="logo", src=app.get_asset_url('DASHLOGO.jpg')),
                html.H4(children="Rate of US Poison-Induced Deaths"),
                html.P(
                    id="description",
                    children="† Deaths are classified using the International Classification of Diseases, \
                    Tenth Revision (ICD–10). Drug-poisoning deaths are defined as having ICD–10 underlying \
                    cause-of-death codes X40–X44 (unintentional), X60–X64 (suicide), X85 (homicide), or Y10–Y14 \
                    (undetermined intent).",
                ),
            ],
        )])
app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 085-441-807
Debugger PIN: 085-441-807
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [14]:
app = dash.Dash()
app.layout = html.Div([html.Div([
        dcc.Graph(figure=fig
        )
    ], style={'width': '100%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(figure=fig2),
        dcc.Graph(figure=fig3),
    ], style={'display': 'inline-block', 'width': '49%'})
                      ])  
app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Debugger PIN: 586-195-095
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
